## Introduction

The Sentence-Window retrieval technique is a simple and effective way to retrieve more context given a user query which matched some document. It is based on the idea that the most relevant sentences are likely to be close to each other in the document. The technique involves selecting a window of sentences around a sentence matching a user query and instead of returning the matching sentence, the entire window is returned. This technique can be particularly useful when the user query is a question or a phrase that requires more context to be understood.

## Haystack Component

The `SentenceWindowRetriever` is the Haystack component that can be used in a Pipeline to implement the Sentence-Window retrieval technique.

`SentenceWindowRetriever(document_store=doc_store, window_size=2)`

The component takes a document_store and a window_size as input. The document_store contains the documents we want to query, and the window_size is used to determine the number of sentences to return around the matching sentence. So the number of sentences returned will be `2 * window_size + 1`. Although we use the term "sentence" as it's inertly attached to this technique, the `SentenceWindowRetriever` actually works with any splitter from the `DocumentSplitter` class, for instance: `word`, `sentence`, `page`.

## Introductory Example

Let's see a simple example of how to use the `SentenceWindowRetriever` in isolation, and later we can see how to use it within a pipeline. We start by creating a document and splitting it into sentences using the `DocumentSplitter` class.

In [5]:
!pip install haystack-ai

In [2]:
from haystack.components.retrievers import SentenceWindowRetriever

In [7]:
from haystack import Document
from haystack.components.preprocessors import DocumentSplitter

splitter = DocumentSplitter(split_length=1, split_overlap=0, split_by="sentence")
text = ("Paul fell asleep to dream of an Arrakeen cavern, silent people all around  him moving in the dim light "
        "of glowglobes. It was solemn there and like a cathedral as he listened to a faint sound—the "
        "drip-drip-drip of water. Even while he remained in the dream, Paul knew he would remember it upon "
        "awakening. He always remembered the dreams that were predictions. The dream faded. Paul awoke to feel "
        "himself in the warmth of his bed—thinking thinking. This world of Castle Caladan, without play or "
        "companions his own age,  perhaps did not deserve sadness in farewell. Dr Yueh, his teacher, had "
        "hinted  that the faufreluches class system was not rigidly guarded on Arrakis. The planet sheltered "
        "people who lived at the desert edge without caid or bashar to command them: will-o’-the-sand people "
        "called Fremen, marked down on no  census of the Imperial Regate.")

doc = Document(content=text)
docs = splitter.run([doc])

this will result in 9 sentences represented as Haystack Document objects. We can then write these documents to a DocumentStore and use the SentenceWindowRetriever to retrieve a window of sentences around a matching sentence.

In [9]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.document_stores.types import DuplicatePolicy

doc_store = InMemoryDocumentStore()
doc_store.write_documents(docs['documents'], policy=DuplicatePolicy.OVERWRITE)

9

Now we use the `SentenceWindowRetriever` to retrieve a window of sentences around a certain sentence. Note that the `SentenceWindowRetriever` receives as input in run time a `Document` present in the document store, and it will rely on the documents metadata to retrieve the window of sentences around the matching sentence. So, one important aspect to notice is that the `SentenceWindowRetriever` needs to be used in conjunction with another `Retriever` that handles the initial user query, such as the `InMemoryBM25Retriever`, and returns the matching documents.

Let's pass the Document containing the sentence `The dream faded.` to the `SentenceWindowRetriever` and retrieve a window of 2 sentences around it. Note that we need to wrap it in a list as the `run` method expects a list of documents.

In [11]:
from haystack.components.retrievers import SentenceWindowRetriever

retriever = SentenceWindowRetriever(document_store=doc_store, window_size=2)
result = retriever.run(retrieved_documents=[docs['documents'][4]])

The result is a dictionary with two keys:

- `context_windows`: a list of strings containing the context windows around the matching sentence.
- `context_documents`: a list of lists of `Document` objects containing the context windows around the matching sentence.

In [14]:
result.keys()

dict_keys(['context_windows', 'context_documents'])

In [15]:
result['context_windows']

[' Even while he remained in the dream, Paul knew he would remember it upon awakening. He always remembered the dreams that were predictions. The dream faded. Paul awoke to feel himself in the warmth of his bed—thinking thinking. This world of Castle Caladan, without play or companions his own age,  perhaps did not deserve sadness in farewell.']

In [16]:
result['context_documents']

[[Document(id=5d093b6ec1a4bdc7e75f033ae0b570e237053213a09b42a56ad815b4d118943d, content: ' Even while he remained in the dream, Paul knew he would remember it upon awakening.', meta: {'source_id': 'b56504f244b7b650096b14d678bc82f3d7fe240bb135361c6a23a14c4b809596', 'page_number': 1, 'split_id': 2, 'split_idx_start': 219}),
  Document(id=4ed71ff61df531053cc7d5f80e8a0bd1e702f3a396f3f3983ceeffe89878a684, content: ' He always remembered the dreams that were predictions.', meta: {'source_id': 'b56504f244b7b650096b14d678bc82f3d7fe240bb135361c6a23a14c4b809596', 'page_number': 1, 'split_id': 3, 'split_idx_start': 303}),
  Document(id=f485258001abdf2deab98249c7f0826b4f6b1bef7c37763d14318e7b595f434f, content: ' The dream faded.', meta: {'source_id': 'b56504f244b7b650096b14d678bc82f3d7fe240bb135361c6a23a14c4b809596', 'page_number': 1, 'split_id': 4, 'split_idx_start': 358}),
  Document(id=f39c29c3a3122affc5909dc7b98f5880d9bd984731380420134c440da6fee363, content: ' Paul awoke to feel himself in the

## Advanced Example

We will use the BBC news dataset to show how the `SentenceWindowRetriever` works with a dataset containing multiple news articles.

### Reading the dataset

The original dataset is available at http://mlg.ucd.ie/datasets/bbc.html, but  it was already preprocessed and stored in
a single CSV file available here: https://raw.githubusercontent.com/amankharwal/Website-data/master/bbc-news-data.csv

In [17]:
from typing import List
import csv
from haystack import Document

def read_documents(file: str) -> List[Document]:
    with open(file, "r") as file:
        reader = csv.reader(file, delimiter="\t")
        next(reader, None)  # skip the headers
        documents = []
        for row in reader:
            category = row[0].strip()
            title = row[2].strip()
            text = row[3].strip()
            documents.append(Document(content=text, meta={"category": category, "title": title}))

    return documents

In [18]:
!wget https://raw.githubusercontent.com/amankharwal/Website-data/master/bbc-news-data.csv

--2024-10-11 15:46:13--  https://raw.githubusercontent.com/amankharwal/Website-data/master/bbc-news-data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5080260 (4.8M) [text/plain]
Saving to: ‘bbc-news-data.csv.2’

bbc-news-data.csv.2 100%[===================>]   4.84M  8.53MB/s    in 0.6s    

2024-10-11 15:46:14 (8.53 MB/s) - ‘bbc-news-data.csv.2’ saved [5080260/5080260]



In [19]:
docs = read_documents("bbc-news-data.csv")

In [20]:
len(docs)

2225

### Indexing the documents

We will now apply the `DocumentSplitter` to split the documents into sentences and write them to an `InMemoryDocumentStore`.

In [21]:
from haystack import Document
from haystack.components.preprocessors import DocumentSplitter
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.document_stores.types import DuplicatePolicy

def index_documents(documents: List[Document]):
    splitter = DocumentSplitter(split_length=1, split_overlap=0, split_by="sentence")
    docs = splitter.run(documents)
    doc_store = InMemoryDocumentStore()
    doc_store.write_documents(docs["documents"], policy=DuplicatePolicy.OVERWRITE)

    return doc_store

### Querying the documents

Let's now build a pipeline to query the documents using the `InMemoryBM25Retriever` and the `SentenceWindowRetriever`.

In [22]:
from haystack import Pipeline
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.retrievers import SentenceWindowRetriever

def querying_pipeline(doc_store: InMemoryDocumentStore, window_size: int = 2):
    pipeline = Pipeline()
    bm25_retriever = InMemoryBM25Retriever(document_store=doc_store)
    sentence_window_retriever = SentenceWindowRetriever(doc_store, window_size=window_size)
    pipeline.add_component(instance=bm25_retriever, name="BM25Retriever")
    pipeline.add_component(instance=sentence_window_retriever, name="SentenceWindowRetriever")
    pipeline.connect("BM25Retriever.documents", "SentenceWindowRetriever.retrieved_documents")

    return pipeline

### Putting it all together

We now read the raw documents, index them, build the querying pipeline, and query the document store for "phishing attacks", returning only the first top most scored document. We also include the outputs from the BM25Retriever
so that we can compare the results with and without the SentenceWindowRetriever.

In [23]:
docs = read_documents("bbc-news-data.csv")
doc_store = index_documents(docs)
pipeline = querying_pipeline(doc_store, window_size=2)
result = pipeline.run(data={'BM25Retriever': {'query': "phishing attacks", "top_k": 1}}, include_outputs_from={'BM25Retriever'})

Let's now inspect the results from the BM25Retriever and the SentenceWindowRetriever. Since we split the documents by sentence, the BM25Retriever returns only the sentence associated with the matching query.

In [24]:
result['BM25Retriever']['documents'][0].content

' The Anti-Phishing Working group reported that the number of phishing attacks against new targets was growing at a rate of 30% or more per month.'

The SentenceWindowRetriever, on the other hand, returns a window of sentences around the matching sentence, giving us more context to understand the sentence.

In [25]:
result['SentenceWindowRetriever']['context_windows'][0]

'"  In particular, phishing attacks, which typically use fake versions of bank websites to grab login details of customers, boomed during 2004. Web portal Lycos Europe reported a 500% increase in the number of phishing e-mail messages it was catching. The Anti-Phishing Working group reported that the number of phishing attacks against new targets was growing at a rate of 30% or more per month. Those who fall victim to these attacks can find that their bank account has been cleaned out or that their good name has been ruined by someone stealing their identity. This change in the ranks of virus writers could mean the end of the mass-mailing virus which attempts to spread by tricking people into opening infected attachments on e-mail messages.'

## Conclusion

We saw how the `SentenceWindowRetriever` works and how it can be used to retrieve a window of sentences around a matching document, give us more context to understand the document. One important aspect to notice is that the `SentenceWindowRetriever` doesn't handle queries directly but relies on the output of another `Retriever` that handles the initial user query. This allows the `SentenceWindowRetriever` to be used in conjunction with any other retriever in the pipeline, such as the `InMemoryBM25Retriever`.